# Ettoday 網路爬蟲實作練習


* 能夠利用 Request + BeatifulSour 撰寫爬蟲，並存放到合適的資料結構


## 作業目標

根據範例：

1. 取出今天所有的新聞
2. 取出現在時間兩小時內的新聞
3. 根據範例，取出三天前下午三點到五點的新聞

## 1. 取出今天所有的新聞

In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
from datetime import datetime
import time

date_time = datetime.now()
date_time = date_time.strftime("%Y/%m/%d")
print(date_time)

browser = webdriver.Chrome(executable_path='chromedriver')
browser.get("https://www.ettoday.net/news/news-list.htm")
html_source = browser.page_source
soup = BeautifulSoup(html_source, "html5lib")

# 每2秒鐘自動往下滑
while True:
    browser.execute_script("window.scrollTo(0, 10000);") # execute_script -> 在webdriver中執行javasript
                            # window.scrollTo(x, y) -> 平滑滾動至給定座標
    time.sleep(2)
    html_source = browser.page_source
    soup = BeautifulSoup(html_source, "html5lib")
    if date_time not in soup.find_all(class_="part_list_2")[-1].find_all('h3')[-1].find(class_="date").text:
        break

In [ ]:
for d in soup.find(class_="part_list_2").find_all('h3'):
    if date_time in d.find(class_="date").text==False:
        break
    print(d.find(class_="date").text, d.find_all('a')[-1].text)

In [ ]:
news = []

for d in soup.find(class_="part_list_2").find_all('h3'):
    if date_time in d.find(class_="date").text :
        break
    news.append({
        '時間': d.find(class_="date").text,
        '標題': d.find_all('a')[-1].text
    })

import pandas as pd
pd.DataFrame(news)

## 2. 取出現在時間兩小時內的新聞

In [ ]:
from datetime import datetime, timedelta
two_hours_ago_time = datetime.now() - timedelta(hours=2)

# 取得資料（整理成好的格式）
news = []

for d in soup.find(class_="part_list_2").find_all('h3'):
    # 比兩小時前早的話就結束
    if datetime.strptime(d.find(class_="date").text, '%Y/%m/%d %H:%M') < two_hours_ago_time:
        break
    
    news.append({
        '時間': d.find(class_="date").text,
        '標題': d.find_all('a')[-1].text
    })

import pandas as pd
pd.DataFrame(news)

## 3. 根據範例，取出三天前下午三點到五點的新聞

In [ ]:
date_time = datetime.now() - timedelta(3)
print(date_time) #三天前日期

date_time_month = date_time.strftime("%m")
date_time_day = date_time.strftime("%d")
print(date_time_month) #月
print(date_time_day) #日


three_days_15_clock = datetime.strptime(date_time.strftime("%Y/%m/%d") + ' 15', '%Y/%m/%d %H')
three_days_17_clock = datetime.strptime(date_time.strftime("%Y/%m/%d") + ' 17', '%Y/%m/%d %H')
print(three_days_15_clock) #下午三點
print(three_days_17_clock) #下午五點

In [ ]:
from selenium.webdriver.support.ui import Select

selM = Select(browser.find_element_by_id("selM"))
selM.select_by_value(date_time_month)
selD = Select(browser.find_element_by_id("selD"))
selD.select_by_value(date_time_day)
browser.find_element_by_id('button').click()